<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/23_AC_Replication/Sargassum_Replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Replicating carbon structure of this paper: https://www.mdpi.com/1420-3049/27/18/6040

In [ ]:
%%capture
!apt-get update
!apt-get install -y build-essential cmake libfftw3-dev libjpeg-dev libpng-dev \
                    libopenmpi-dev openmpi-bin python3-dev python3-numpy git

# Clone the LAMMPS repository
%cd /content
!git clone -b stable https://github.com/lammps/lammps.git
%cd lammps

# Create a build directory and compile LAMMPS with required packages
!mkdir build
%cd build
!cmake ../cmake -DBUILD_SHARED_LIBS=yes \
                -DLAMMPS_EXCEPTIONS=yes \
                -DPKG_MOLECULE=yes \
                -DPKG_KSPACE=yes \
                -DPKG_RIGID=yes \
                -DPKG_MANYBODY=yes \
                -DPKG_USER-MISC=yes \
                -DPKG_PYTHON=yes \
                -DPYTHON_EXECUTABLE=`which python3`
!make -j4
!make install-python

# Return to the working directory
%cd /content/


In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "4"  # limit OpenMP threads if you want

import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.constants import Avogadro

!pip install MDAnalysis
import MDAnalysis as mda
from MDAnalysis.analysis.rdf import InterRDF

# We can also create a symlink so we can call "lmp" directly:
!ln -s /content/lammps/build/lmp /usr/local/bin/lmp
!which lmp


In [ ]:
def create_AC_23rings():
    """
    Creates a single ~23-ring aromatic flake with:
      - 2 carboxylate groups (-COO-),
      - 4 carbonyl (C=O),
      - 8 hydroxyl (-OH),
    following the approach used in the paper for SAC600_3/1.

    Returns:
      positions (list of [x,y,z]),
      bonds (list of (bond_id, bond_type, atom1, atom2)),
      angles (list of angle_tuples),
      atom_info (list of dicts with 'type', 'charge' fields, etc.)
    """
    positions = []
    bonds = []
    angles = []
    atom_info = []

    # We'll just place 46 carbons in a planar circle (placeholder).
    N = 46
    R = 5.0
    bond_id = 1

    # 1) Place carbons
    for i in range(N):
        angle = 2.0 * np.pi * i / N
        x = R * np.cos(angle)
        y = R * np.sin(angle)
        z = 0.0
        positions.append([x, y, z])
        atom_info.append({'elem': 'C', 'type': 1, 'charge': 0.0})

    # 2) Connect them in a ring
    for i in range(N):
        j = (i + 1) % N
        bonds.append((bond_id, 1, i+1, j+1))
        bond_id += 1

    # 3) 2 carboxylate groups => each has 2 O atoms
    chosen_carbon_ids_carbox = [1, 2]
    for c_id in chosen_carbon_ids_carbox:
        cpos = np.array(positions[c_id-1])
        direction = cpos / np.linalg.norm(cpos)
        Opos1 = cpos + direction * 1.2
        Opos2 = cpos + direction * 1.4
        posO1_idx = len(positions) + 1
        posO2_idx = len(positions) + 2
        positions.append(Opos1.tolist())
        positions.append(Opos2.tolist())

        # bond to carbon
        bonds.append((bond_id, 3, c_id, posO1_idx))
        bond_id += 1
        bonds.append((bond_id, 3, c_id, posO2_idx))
        bond_id += 1

        atom_info[c_id-1]['charge'] += 0.4
        atom_info.append({'elem': 'O', 'type': 2, 'charge': -0.7})
        atom_info.append({'elem': 'O', 'type': 2, 'charge': -0.7})

    # 4) 4 carbonyl (C=O)
    chosen_carbonyl_ids = [3,4,5,6]
    for c_id in chosen_carbonyl_ids:
        cpos = np.array(positions[c_id-1])
        direction = cpos / np.linalg.norm(cpos)
        Opos = cpos + direction * 1.2
        posO_idx = len(positions) + 1
        positions.append(Opos.tolist())
        bonds.append((bond_id, 3, c_id, posO_idx))
        bond_id += 1

        atom_info[c_id-1]['charge'] += 0.5
        atom_info.append({'elem': 'O', 'type': 2, 'charge': -0.5})

    # 5) 8 hydroxyl (–OH)
    chosen_hydroxyl_ids = [7,8,9,10,11,12,13,14]
    for c_id in chosen_hydroxyl_ids:
        cpos = np.array(positions[c_id-1])
        direction = cpos / np.linalg.norm(cpos)
        Opos = cpos + direction * 1.2
        posO_idx = len(positions) + 1
        positions.append(Opos.tolist())
        bonds.append((bond_id, 3, c_id, posO_idx))
        bond_id += 1

        Hpos = Opos + direction * 0.96
        posH_idx = len(positions) + 1
        positions.append(Hpos.tolist())
        bonds.append((bond_id, 2, posO_idx, posH_idx))
        bond_id += 1

        atom_info[c_id-1]['charge'] += 0.3
        atom_info.append({'elem': 'O', 'type': 2, 'charge': -0.5})
        atom_info.append({'elem': 'H', 'type': 3, 'charge': 0.2})

    return positions, bonds, angles, atom_info


In [ ]:
def setup_and_run_sim(num_Hg=5, box_size=(50.0, 50.0, 50.0),
                      num_Mg=2, num_Zn=2, num_Ca=2, num_water=500):
    """
    Creates data_{num_Hg}.lmp with the specified AC + ions/water,
    then runs a short 2000-step LAMMPS simulation to test stability.
    """

    data_file = f"data_{num_Hg}.lmp"
    dump_file = f"dump_{num_Hg}.lammpstrj"

    # 1) Build the data file
    create_data_file(num_Hg=num_Hg, filename=data_file, box_size=box_size,
                     num_Mg=num_Mg, num_Zn=num_Zn, num_Ca=num_Ca, num_water=num_water)

    # 2) Write LAMMPS input script
    input_script = f"""
units           real
atom_style      full
boundary        p p p
read_data       {data_file}

pair_style      lj/cut/coul/long 12.0 12.0
kspace_style    pppm 1.0e-5

pair_coeff 1 1 0.07 3.40  # C
pair_coeff 2 2 0.16 3.05  # O
pair_coeff 3 3 0.02 0.00  # H
pair_coeff 4 4 0.20 3.70  # Hg2+
pair_coeff 5 5 0.10 3.40  # Cl
pair_coeff 6 6 0.15 2.47  # Mg2+
pair_coeff 7 7 0.20 2.90  # Zn2+
pair_coeff 8 8 0.15 3.47  # Ca2+
pair_modify     mix arithmetic

bond_style      harmonic
bond_coeff      1 450.0 1.42
bond_coeff      2 450.0 0.9572
bond_coeff      3 450.0 1.43

angle_style     harmonic
angle_coeff     1 55.0 120.0
angle_coeff     2 55.0 104.52

group           carbon type 1
fix             fix_carbon carbon setforce 0.0 0.0 0.0

group           water type 2 3
fix             fix_shake water shake 0.0001 20 0 b 2 a 2

neighbor        2.0 bin
neigh_modify    delay 0 every 1 check yes

min_style       cg
minimize        1e-4 1e-6 100 1000

velocity        all create 300.0 12345 mom yes rot yes dist gaussian
fix             nvt_control all nvt temp 300.0 300.0 100.0

thermo          1000
thermo_style    custom step temp etotal press

dump            1 all atom 100 {dump_file}

timestep        1.0
run             2000
"""

    # Write it out
    with open("in_run.lmp", "w") as f:
        f.write(input_script)

    # 3) Execute LAMMPS
    os.system(f"lmp -in in_run.lmp")


In [ ]:
def analyze_simulation(num_Hg=5, box_dims=(50,50,50), mass_adsorbent=1.0,
                       initial_counts=None):
    """
    Reads dump_{num_Hg}.lammpstrj + data_{num_Hg}.lmp,
    computes RDF of carbon(type 1) vs mercury(type 4),
    and a simplistic "adsorption" measure (avg # of Hg near carbon).
    """
    if initial_counts is None:
        initial_counts = {'Hg': num_Hg, 'Mg': 2, 'Zn': 2, 'Ca': 2}

    data_file = f"data_{num_Hg}.lmp"
    dump_file = f"dump_{num_Hg}.lammpstrj"

    if not os.path.exists(dump_file):
        print(f"Dump file {dump_file} not found. Skipping analysis.")
        return None, None, None, None, None

    # Load with MDAnalysis
    u = mda.Universe(
        topology=data_file,
        trajectory=dump_file,
        topology_format='DATA',
        trajectory_format='LAMMPSDUMP'
    )

    ref_group = u.select_atoms('type 1')  # carbon
    target_group = u.select_atoms('type 4')  # mercury

    # RDF
    r_min, r_max = 0.0, 12.0
    rdf_calc = InterRDF(ref_group, target_group, range=(r_min, r_max), nbins=120)
    rdf_calc.run()

    r = rdf_calc.bins   # .results.bins in newer versions
    rdf = rdf_calc.rdf  # .results.rdf in newer versions

    # find a cutoff from the first local minima
    from scipy.signal import find_peaks
    peaks, _ = find_peaks(rdf)
    if len(peaks) > 0:
        first_peak = peaks[0]
        inverted = -rdf
        minima, _ = find_peaks(inverted, distance=10)
        cands = minima[minima > first_peak]
        if len(cands) > 0:
            r_cut = r[cands[0]]
        else:
            r_cut = r_max
    else:
        r_cut = r_max

    # measure how many mercury are near carbon
    from scipy.spatial import cKDTree
    adsorbed = 0
    total_frames = 0
    for ts in u.trajectory:
        p_ref = ref_group.positions
        p_tg = target_group.positions
        tr_ref = cKDTree(p_ref)
        sdm = tr_ref.sparse_distance_matrix(cKDTree(p_tg), max_distance=r_cut, output_type='coo_matrix')
        col_ids = np.unique(sdm.col)
        adsorbed += len(col_ids)
        total_frames += 1

    avg_adsorbed = adsorbed / total_frames if total_frames > 0 else 0.0

    # compute q_e
    MW_Hg = 200.59
    q_e = (avg_adsorbed * MW_Hg) / mass_adsorbent * 1e3  # mg/g

    # compute Ce
    volume_A3 = box_dims[0]*box_dims[1]*box_dims[2]
    volume_L = volume_A3 * 1e-24
    from scipy.constants import Avogadro
    N_initial = initial_counts['Hg']
    N_unads = N_initial - avg_adsorbed
    C_e = (N_unads * MW_Hg / Avogadro) / volume_L * 1e3  # mg/L

    return C_e, q_e, r, rdf, r_cut


In [ ]:
import csv

# Let’s do a quick test with multiple Hg_counts
Hg_counts = [5, 10, 15, 20]

Ce_values = []
qe_values = []
rdf_data = []

for Hg_num in Hg_counts:
    # 1) Run a short 2000-step sim
    setup_and_run_sim(
        num_Hg=Hg_num,
        box_size=(50,50,50),
        num_Mg=2,
        num_Zn=2,
        num_Ca=2,
        num_water=500
    )

    # 2) Analyze
    Ce, q_e, r, rdf, r_cut = analyze_simulation(Hg_num, box_dims=(50,50,50),
                                                initial_counts={'Hg': Hg_num, 'Mg':2, 'Zn':2, 'Ca':2})
    if Ce is not None:
        Ce_values.append(Ce)
        qe_values.append(q_e)
        rdf_data.append((Hg_num, r, rdf))

# Save to CSV
with open('sorption_data.csv','w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Hg_num","Ce(mg/L)","q_e(mg/g)"])
    for num, Ce, q_e in zip(Hg_counts, Ce_values, qe_values):
        writer.writerow([num, Ce, q_e])

# If we have enough data, attempt a Langmuir fit
if len(Ce_values) > 2:
    def langmuir(C, q_max, K_L):
        return (q_max*K_L*C)/(1.0 + K_L*C)

    try:
        params, _ = curve_fit(langmuir, Ce_values, qe_values, p0=[max(qe_values),0.1])
        Ce_fit = np.linspace(min(Ce_values), max(Ce_values), 100)
        qe_fit = langmuir(Ce_fit,*params)

        plt.figure(figsize=(7,5))
        plt.scatter(Ce_values, qe_values, c='b', label='Data')
        plt.plot(Ce_fit, qe_fit, 'r--', label='Langmuir fit')
        plt.xlabel("Ce (mg/L)")
        plt.ylabel("qe (mg/g)")
        plt.title("Hg Sorption Isotherm")
        plt.grid(True)
        plt.legend()
        plt.show()

        print("Langmuir fit parameters:")
        print(f"q_max= {params[0]:.2f}, K_L= {params[1]:.4f}")
    except Exception as e:
        print("curve_fit failed:", e)

# Plot RDF from each simulation
plt.figure(figsize=(8,6))
for (Hg_num, r, rdf) in rdf_data:
    plt.plot(r, rdf, label=f"Hg={Hg_num}")
plt.xlabel("Distance (Å)")
plt.ylabel("g(r)")
plt.title("Carbon-Hg RDF")
plt.legend()
plt.grid(True)
plt.show()
